In [1]:

using Pipe
# using MarketTechnicals
# using Plots
# using TimeSeries
using DataFrames
# using StatsPlots
using CSV
using RollingFunctions
using Query
# using GLMakie

using JSON
using DataFrames
using Dates
using Colors
using ColorSchemes
using Indicators

import Colors
using Plots
using Interact

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-13141021074721894124\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-13936065995727558201\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-13141021074721894124\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
json_data_folder = "../user_data/data/binance"
json_filename = "BTC_USDT-15m.json"
json_filepath = "$json_data_folder/$json_filename"

"../user_data/data/binance/BTC_USDT-15m.json"

In [3]:
f = open(json_filepath, "r")
json_data = JSON.parse(f)
close(f)

In [4]:
dfa = DataFrame(unix_time = Int64[], open=Float64[], high=Float64[], low=Float64[], close=Float64[], volume=Float64[])
for r in json_data push!(dfa, r) end
dfa.unix_time .= dfa.unix_time ./1000
# dfa.time .= unix2datetime.(dfa.unix_time)

insertcols!(dfa, :time => unix2datetime.(dfa.unix_time))
1

1

In [1501]:
start_date = DateTime(2021, 1, 1)
end_date = DateTime(2021, 5, 24)
df = dfa[(dfa.time .>= start_date) .& (dfa.time .<= end_date) , :]

ndfrows = nrow(df)

13690

In [1502]:
# df.low

In [663]:
cleanup_trades(df) = begin
    ndfrows = nrow(df)
    df.reason .= [fill("", (ndfrows,1))...]
    df.price_in_long .= [fill(0., (ndfrows,1))...]
    df.price_out_long .= [fill(0., (ndfrows,1))...]
    df.stoploss_long .= [fill(0., (ndfrows,1))...]
    df.buy_long .= [fill(0., (ndfrows,1))...]
    df.sell_long .= [fill(0., (ndfrows,1))...]
    df.pnl .= [fill(0., (ndfrows,1))...]
    df.capital .= [fill(0., (ndfrows,1))...]
end
1

1

In [687]:
# inng_sigs = map(i -> (x_c_cema[i] < q_undervalue) & (x_lema_lemalag[i] > q_trend), inxs)
# df.high

In [708]:

    ps = psar([df.high df.low])
    
    long_sigs = map(i -> i < 30 ? false : df.close[i] >= ps[i] && df.close[i-3] < ps[i] , 1:length(df.high))
    filter(x -> x, long_sigs)
1

1

In [684]:
# [ps df.close][1]

# filter( x -> x[1] > x[2],  [ps df.close])

In [730]:
dft = DataFrame(unix_time = Int64[], time = DateTime[],
        reason = String[],
        price_in_long = Float64[],
        price_out_long = Float64[],
        buy_long_qty = Float64[],
        sell_long_qty = Float64[],
        pnl = Float64[],
        capital = Float64[],
    )

,unix_time,time,reason,price_in_long,price_out_long,buy_long_qty,sell_long_qty
,Int64,DateTime,String,Float64,Float64,Float64,Float64


In [731]:
# backtest(q_undervalue,q_trend) = begin
#     xxx::Array{Float64}
#     ema_len = 14
#     q_undervalue,q_trend = float(xxx[1]), float(xxx[2]) #, xxx[3]
    
    ndfrows = nrow(df)
    inxs = 1:ndfrows
#     datr = atr(df.close)
    ps = psar([df.high df.low])

    long_sigs = map(i -> i < 30 ? false : df.close[i] >= ps[i] && df.close[i-3] < ps[i] , 1:length(df.high))
    
    capital, risk_percent = 100., 0.02
    isLong, price_in_long, price_out_long, buy_long_qty, pnl, stop_long_price  = false, 0, 0, 0, 0, 0 
    time_gone_long = 0
    fee_percent = 0.001

    for i in 8:ndfrows
        r = df[i,:]
#         (high,low,close,ratr,ttime) = (hlc[i,1], hlc[i,2], hlc[i,3],datr[i],time[i])
        (high,low,close,ttime) = (r.high,r.low,r.close, r.time)
        if isLong        
#         print("islong $close / $price_in_long ")
            do_sell = close >= price_in_long * (1. + 0.01) #|| close < stop_long_price
               
        
            if close < stop_long_price || do_sell # close SL            
                price_out_long = close * (1 - fee_percent)
                pnl = buy_long_qty * (price_out_long - price_in_long)
                capital += pnl
                isLong = false
                
                push!(dft, Dict(:unix_time => unix_time[i],
                    :time => ttime,
                    :reason => "STOP LONG",
                    :price_in_long => price_in_long,
                    :price_out_long => price_out_long,
                    :buy_long_qty => 0,
                    :sell_long_qty => buy_long_qty,
                    :pnl => pnl,
                    :capital =>capital
                ))
            end
        elseif long_sigs[i]
#             print("not long")   
                isLong = true
                position = capital * risk_percent
                price_in_long = close * (1 + fee_percent)
                stop_long_price = close * (1 - 0.03)

                buy_long_qty = position / (price_in_long - stop_long_price)
                
                
                push!(dft, Dict(:unix_time => unix_time[i],
                    :time => ttime,
                    :reason => "GO LONG",
                    :price_in_long => price_in_long,
                    :price_out_long => 0,
                    :buy_long_qty => buy_long_qty,
                    :sell_long_qty => 0,
                    :pnl => 0,
                    :capital =>capital
                ))
                
                
                time_gone_long = ttime
        end
    end

CSV.write("trades.csv", dft)
    capital
# long_sigs
# end

87.22707643280708

In [341]:
# using Blink
# w = Window()
body!(w, p);

In [227]:
# CSV.write("trades.csv", trades)
dft |>
    @groupby(_.pnl >= 0) |>
    @map({Key=key(_) > 0 ? "profit" : "loss", count = length(_), sum=sum(_.pnl)}) |>
    DataFrame
# dft


,Key,count,sum
,String,Int64,Float64
1,profit,252,73.1746
2,loss,52,-99.7535


In [447]:
start_date = DateTime(2021, 5, 6)
end_date = DateTime(2021, 5, 9)
vdf = dfa[(dfa.time .>= start_date) .& (dfa.time .<= end_date) , :]
# hlc = [vdf.high vdf.low vdf.close]
# hh, ll, cc = hlc[:,1],hlc[:,2],hlc[:,3]
tt, hh, ll, cc = vdf.time, vdf.high, vdf.low, vdf.close
# h14max = [fill(vdf[1,:high],8-1)...,rollmax(vdf.high, 8)...]
#  length(h14max), length(vdf.close)
1

1

In [553]:



# l = @layout [a{0.8w}; c{0.2h}]

mp = @manipulate for nh in slider(1:1:200; label="nh", default=14),
    pullback in slider(-5:0.05:0; label="pullback", default=-0.5),
    fwd in slider(-100:1:100; label="fwd", default=14)
#     phmax in slider(-5:0.2:5; label="phmax", default=1.02)
# phmax = 1.02
#     ema_len in slider(1:1:200; label="ema len", default=1),
#     q_undervalue in slider(-10:0.05:10; label="q_undervalue", default=1),
#     q_trend in slider(-20:0.05:20; label="q_trend", default=1)
        
#     cema = ema(vdf.close, n=ema_len)
#     lema = ema(vdf.low, n=ema_len)
#     lemalag = circshift(lema,14)
#     x_c_cema = log.(vdf.close ./ cema) .* 100
#     x_lema_lemalag = log.(lema ./ lemalag) .* 100
    
   
    hmax = [fill(hh[nh-1],nh-1)..., rollmax(hh, nh)...]
    fwhmax = circshift(hmax, fwd)
    
    buy_filters = (cc .< ( cc .* (1 + pullback / 100.))) # .& (cc .> hh .* (1-phmax))
    buy_sig_indxs = filter(i -> buy_filters[i] , 1:length(tt))
    buy_sigs = map(i -> tt[i], buy_sig_indxs)
    
#     buy_sig_indxs = filter(i -> (x_c_cema[i] < q_undervalue) & (x_lema_lemalag[i] > q_trend), inxs)
#     lema = ema( vdf.low, n=ema_len)

    p1 = plot(tt, [cc, fwhmax .* (1 + pullback / 100.) ], labels = ["close" "fwhmax"], size=(1600, 800), legend = :outertopright)
    
#     plot!( tt, fwhmax .* (1 + pullback / 100.))
#     plot!(p1[1], vdf.time, [cema, lemalag], labels = [ "cema($ema_len)" "lemalag($ema_len)"])
#     plot!(p1[1], vdf.time, [vdf.close .+ _atr], ["close + atr"], lw=0.3)
#     if length(buy_sigs) > 0 vline!(p1[1], buy_sigs, lw=0.5, color=:green) end
#       vline!(p1[1], buy_sigs, lw=0.5, color=:green)

end
#     p2 = plot(x, [x_lema_lemalag, x_c_cema], labels = ["x_lema_lemalag" "x_c_cema"])
    

#     plot(p1, p2, layout = l, size=(1000, 600), legend = :outertopright)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["nh"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 200, :default => 14, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :min => 1, :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Any} with 2 listeners. Value:
100, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/johnsmith/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/johnsmith/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/johnsmith/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), Asset("css", nothing, "/Users/johnsmith/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), Asset("css", nothing, "/Users/johnsmith/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000014dc93420, Task (runnable) @0x000000014dc93420), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\",\"21\",\"22\",\"23\",\"24\",\"25\",\"26\",\"27\",\"28\",\"29\",\"30\",\"31\",\"32\",\"33\",\"34\",\"35\",\"

In [175]:
mp = @manipulate for ema_len in slider(1:1:200; label="ema len", default=1)
#     q_undervalue in slider(-10:0.05:10; label="q_undervalue", default=1),
#     q_trend in slider(-20:0.05:20; label="q_trend", default=1)
        
    cema = ema(cc, n=ema_len)
    lema = ema(ll, n=ema_len)
    lemalag = circshift(lema,14)
    x_c_cema = log.(cc ./ cema) .* 100
    x_lema_lemalag = log.(lema ./ lemalag) .* 100
    
   
#     hmax = [fill(hh[nh-1],nh-1)..., rollmax(hh, nh)...]
#     fwhmax = circshift(hmax, fwd)    
#     buy_filters = (cc .< ( cc .* (1 + pullback / 100.))) # .& (cc .> hh .* (1-phmax))
#     buy_sig_indxs = filter(i -> buy_filters[i] , 1:length(tt))
#     buy_sigs = map(i -> tt[i], buy_sig_indxs)
    
#     buy_sig_indxs = filter(i -> (x_c_cema[i] < q_undervalue) & (x_lema_lemalag[i] > q_trend), inxs)
#     lema = ema( vdf.low, n=ema_len)

    p1 = plot(tt, [cc], labels = ["close" ], size=(1600, 800), legend = :outertopright)
    
    plot!(p1[1], tt, [lema, lemalag], labels = [ "lema($ema_len)" "lemalag($ema_len)"])
#     plot!(p1[1], vdf.time, [vdf.close .+ _atr], ["close + atr"], lw=0.3)
#     if length(buy_sigs) > 0 vline!(p1[1], buy_sigs, lw=0.5, color=:green) end
#       vline!(p1[1], buy_sigs, lw=0.5, color=:green)

end
#     p2 = plot(x, [x_lema_lemalag, x_c_cema], labels = ["x_lema_lemalag" "x_c_cema"])
    

#     plot(p1, p2, layout = l, size=(1000, 600), legend = :outertopright)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["ema len"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 200, :default => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :min => 1, :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Any} with 2 listeners. Value:
100, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/johnsmith/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/johnsmith/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/johnsmith/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), Asset("css", nothing, "/Users/johnsmith/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), Asset("css", nothing, "/Users/johnsmith/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000014302bdb0, Task (runnable) @0x000000014302bdb0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\",\"21\",\"22\",\"23\",\"24\",\"25\",\"26\",\"27\",\"28\",\"29\",\"30\",\"31\",\"32\",\"33\",\"34\",\"35\

steps! (generic function with 2 methods)

In [462]:
find_extrema(ar, idxs) = begin # returns indexes in original array ar
#     ar = cc
    llen = length(ar)
    hids = Int64[1]
    lids = Int64[1]
    parhids = zeros(Int64, llen)
    parlids = zeros(Int64, llen)
    parhids[1] = parlids[1] = 1
    push!(hids,1)
    push!(lids,1)
    up = true

    for i in 2:llen
        if up && ar[i] < ar[i-1]
            push!(hids, idxs[i-1])
            up = false
        elseif !up && ar[i] > ar[i-1]
            push!(lids,idxs[i-1])
            up = true
        end
        parhids[i] = last(hids)
        parlids[i] = last(lids)
    end
    
#     parhids = [last(filter(idx -> idx <= i, hids)) for i in fillrange]
#     parlids = [last(filter(idx -> idx <= i, lids)) for i in fillrange]
    
    (hids, lids, parhids, parlids)
end

find_extrema (generic function with 3 methods)

1

In [5]:
start_date = DateTime(2021, 3, 6)
end_date = DateTime(2021, 3, 8)
vdf = dfa[(dfa.time .>= start_date) .& (dfa.time .<= end_date) , :]
utt, tt, hh, ll, cc = vdf.unix_time, vdf.time, vdf.high, vdf.low, vdf.close
# h14max = [fill(vdf[1,:high],8-1)...,rollmax(vdf.high, 8)...]
#  length(h14max), length(vdf.close)
# hlc = [hh ll cc]
length(tt)

187

In [174]:
movements(ar) = [
    [true, ((ar[1:length(ar)-2] .< ar[2:length(ar)-1]) .& (ar[2:length(ar)-1] .> ar[3:length(ar)]))..., false], #hids
    [true, ((ar[1:length(ar)-2] .> ar[2:length(ar)-1]) .& (ar[2:length(ar)-1] .< ar[3:length(ar)]))..., false], #lids
    [true, ((ar[1:length(ar)-2] .< ar[2:length(ar)-1]) .& (ar[2:length(ar)-1] .< ar[3:length(ar)]))..., false], #ups
    [true, ((ar[1:length(ar)-2] .> ar[2:length(ar)-1]) .& (ar[2:length(ar)-1] .> ar[3:length(ar)]))..., false]  #dns
]

getTrueIndxs(arr::Vector{Bool}) = [i for i in 1:length(arr) if arr[i]]

expandl(a) = foldl(append!, [fill(a[i-1], a[i]-a[i-1]) for i in 2:length(a)], init=[])

steps!(x,y, stp; args...) = begin
        plot!(
            vec([x[stp[1:length(stp)-1]] x[stp[2:length(stp)]]  x[stp[2:length(stp)]] fill(NaN,length(stp)-1)]'),
            vec([y[stp[1:length(stp)-1]] y[stp[1:length(stp)-1]] y[stp[2:length(stp)]] fill(NaN,length(stp)-1)]')
        ; args...)

end

ar = cc
ut = utt
llen = length(ar)
irs = 1:llen
ema = Indicators.ema
# ema26 = ema(ar,n=26)
# ema50 = ema(ar,n=50)
# ema100 = ema(ar,n=100)
# catr = Indicators.atr(hlc, n=14)

##### hids and lids
npopulate(ar) = [i for i in 1:length(ar) if ar[1][i]]

hlud = movements(ar)
(hids,lids,ups,dns) = getTrueIndxs.(hlud)
(xphids,xplids,xpups,xpdns) = expandl.([hids,lids,ups,dns])

hidshlud = movements(ar[hids])
(hids2hids,lids2hids,ups2hids,dns2hids) = getTrueIndxs.(hidshlud)
xphids2hids = @pipe hids2hids |> hids[_] |> expandl(_)
xplids2hids = @pipe lids2hids |> hids[_] |> expandl(_)

lidshlud = movements(ar[lids])
(hids2lids,lids2lids,ups2lids,dns2lids) = getTrueIndxs.(lidshlud)
xplids2lids = @pipe lids2lids |> lids[_] |> expandl(_)

##### emas

# ema26hlud = movements(ema26)
# ema26hids = [i for i in 2:length(ema26)-1 if ema26hlud[1][i-1]]
# ema26lids = [i for i in 2:length(ema26)-1 if ema26hlud[2][i-1]]


# (ema26hids, ema26lids, ema26parhids, ema26parlids) = find_extrema(ema26, 1:llen)

# (ema26hids2lids, ema26lids2lids, ema26parhids2lids, ema26parlids2lids) = find_extrema(ema26lids, 1:llen)

# (ema50hids, ema50lids, ema50parhids, ema50parlids) = find_extrema(ema50, 1:llen)

# (ema50hids2lids, ema50lids2lids, ema50parhids2lids, ema50parlids2lids) = find_extrema(ema50lids, 1:llen)
##### buy signals

# long_sig_utidxs = [i for i in 21:llen if 
#             ema26[i] > ema50[i] &&
# #             ar[i] < ar[parhids[i]] && 
#             ema26parhids[i] < ema50parhids[i]
# #             ar[i] > ar[parhids2hids[i]] && ar[i - 1] < ar[parhids2hids[i]]
# #             isnothing(findfirst(ar[i-10:i-3] .> ar[plphids[i-10:i-3]] * 1.005)) #&&
# #             ar[i] < ar[i-1]
#         ]
            
# long_sig_utidxs = lids[long_sig_lididxs]
# stops = ar[long_sig_utidxs] * (1 - 0.01)



# f = plot(ut, ar, size=(1500,600),lw=0.5, legend = :outertopright, labels="close")



# using Blink
# w = Window()
# body!(w, f);
# f
1

1

In [16]:
# check
ar = cc
ut = utt
ih, il = Int64[1], Int64[1]
# qp = Vector{Tuple{Int64,Float64}}()
# PricePoint = Tuple{Int64,Float64}
# PricePoint = @NamedTuple begin
#            t::Int64
#            p::Float64
#        end

# QPriceHighLows = @NamedTuple begin
#     d::Array{PricePoint}
#     l::QPriceHighLows
#     h::QPriceHighLows
# end

# PricePointArray = AbstractArray{PricePoint}
# struct QPrices
#     :d::PricePoint[]
    
# end

# QHighLow = Tuple{PricePointArray,PricePointArray}

qq = (d = [],
        l = (d = [],
                    l = (d = []),
                    h = (d = [])),
        h = (d = [],
                    l = (d = []),
                    h = (d = [])))

# qq = Dict(:d => PricePoint[],
#         :l => Dict(:d => PricePoint[],
#                     :l => Dict(:d => PricePoint[]),
#                     :h => Dict(:d => PricePoint[])),
#         :l => Dict(:d => [],
#                     :l => Dict(:d => PricePoint[]),
#                     :h => Dict(:d => PricePoint[]))
# )

# qq = QPriceHighLows(d = [],
#     h = QPriceHighLows(d = [])
# )

pushq(q, pp::Tuple{Int64,Float64}, lvl::Int) = begin
        println(lvl)
#      println(q)
    !haskey(q,:d) && return
    push!(q[:d], pp)
#     ar, l,h = qp
    len = length(q[:d])
    if len < 3 return end
    
    #low i-1
    if lvl > 0 && haskey(q,:l) && q[:d][len-2].p > q[:d][len-1].p && q[:d][len-1].p < q[:d][len].p 
        println("go left")
        populateHL(q[:l], pp, lvl - 1);
    end
    
#     haskey(q,:h) && q.d[len-2] < q.d[len-1] && q.d[len-1] > q.d[len] && #high i-1
#        populateHL(q.h, pp);
    
end

for i in 3:length(ar)
    
    pushq(qq, (t=ut[i], p=ar[i]), 3)
#     len = length(qp)
    
end

# f = plot(ut, ar, size=(1500,600),lw=0.5, legend = :outertopright, labels="close")
# scatter!(ut[il], ar[il])
# scatter!(ut[ih], ar[ih])

LoadError: MethodError: no method matching pushq(::NamedTuple{(:d, :l, :h), Tuple{Vector{Any}, NamedTuple{(:d, :l, :h), Tuple{Vector{Any}, Vector{Any}, Vector{Any}}}, NamedTuple{(:d, :l, :h), Tuple{Vector{Any}, Vector{Any}, Vector{Any}}}}}, ::NamedTuple{(:t, :p), Tuple{Int64, Float64}}, ::Int64)
[0mClosest candidates are:
[0m  pushq(::Any, [91m::Tuple{Int64, Float64}[39m, ::Int64) at In[16]:47

In [1491]:
    

f1 = plot(tt, cc, linewidth=1, color=:blue, size=(1200,600))
# f1 = plot(tt, hh, linewidth=1, color=:green, size=(1200,600), lw=0.3, label="high")
# plot!(tt, ll, linewidth=1, color=:red, size=(1200,600), lw=0.3, label="price")

plot!(tt, mpk, linewidth=.7, color=:red, label="peaks")
# plot!(tt, mtr, linewidth=.7, color=:green, label="bottoms")

# plot!(tt, csma, linewidth=1, color=:orange, label="csma")
# plot!(hh, linewidth=.5, color=:orange)
scatter!(spktt, spkpp, color=:green, markersize=2, label=nothing)
# scatter!(strtt, strpp, color=:red, markersize=2, label=nothing)
# f2 = plot(m, linewidth=1, color=[:green :cyan :orange])
# hline!([0.0], linestyle=:dash, color=:grey, label="")
# f3 = plot(r, linewidth=1, color=:gold)
# hline!([20, 80], linestyle=:dot, color=[:green, :red], label="")
# fig = plot(f1, f2, f3, layout=@layout[a{0.6h}; b{0.2h}; c{0.2h}], legend = :outertopright, size=(1200,600))
# f1
using Blink
# w = Window()
body!(w, f1);

In [235]:
# ohlc = [df.open df.high df.low df.close]

# ohlc
# import Pkg; Pkg.add("Optim")
using Optim

ff(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
x0 = [0.0, 0.0]
res = optimize(ff, x0)

 * Status: success

 * Candidate solution
    Final objective value:     3.525527e-09

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    60
    f(x) calls:    117
